In [9]:
import tensorflow as tf 
import tensorflow.keras as keras 
import numpy as np 

In [10]:
class Mymodel(keras.Model):
    def __init__(self):
        super(Mymodel, self).__init__()
        self.CRNN = keras.Sequential([
            keras.layers.Conv2D(filters=64, kernel_size=3,activation='relu', padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Conv2D(filters=64, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Conv2D(filters=64, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Conv2D(filters=64, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Reshape(target_shape=(2*10, 64)),
            keras.layers.LSTM(128, return_sequences=False),
            keras.layers.Dense(10, activation='softmax')
            
        ])
    def call(self,x):
        return self.CRNN(x)

In [11]:
def load_dataset(path):
    X = []
    for i in range(1,10):
        data = np.load(path+'dataX'+ str(i) +'.npz')
        X.append(data['arr'])
    Y = np.load(path+'dataY.npz')
    Y = np.array(Y['arr'])
    X1 = X[0]
    for i in range(1,len(X)):
        X1 = np.concatenate((X1,X[i]))
    return X1,Y

In [12]:
X,Y=load_dataset('data/')
print(X.shape, Y.shape)

(8732, 40, 173) (8732, 10)


In [13]:
## shuffle the datasets, makes train better 
idx  = np.arange(8732)
np.random.shuffle(idx)
## [0,6986） train  80% ; [6986，8732) test 20%
Xs,Ys = X[idx],Y[idx]
trainX, testX = Xs[0:6986],Xs[6986:8732]
trainY, testY = Ys[0:6986],Ys[6986:8732]
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)
# reshape the train data
trainX = trainX.reshape((-1,40,173,1))
testX = testX.reshape((-1,40,173,1))
print(trainX.shape, testX.shape)

(6986, 40, 173) (6986, 10)
(1746, 40, 173) (1746, 10)
(6986, 40, 173, 1) (1746, 40, 173, 1)


In [14]:
## build the model
model=Mymodel()
model.compile(optimizer=keras.optimizers.Adam(),
              loss = keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [15]:
## fit the model
model.fit(trainX, trainY, batch_size=64, epochs=10)

Train on 6986 samples
Epoch 1/10
6986/6986 [==============================] - 73s 10ms/sample - loss: 1.8553 - accuracy: 0.2916
Epoch 2/10
6986/6986 [==============================] - 71s 10ms/sample - loss: 1.4261 - accuracy: 0.4668
Epoch 3/10
6986/6986 [==============================] - 71s 10ms/sample - loss: 1.2541 - accuracy: 0.5369
Epoch 4/10
6986/6986 [==============================] - 71s 10ms/sample - loss: 1.1627 - accuracy: 0.5754
Epoch 5/10
6986/6986 [==============================] - 71s 10ms/sample - loss: 1.0519 - accuracy: 0.6195
Epoch 6/10
6986/6986 [==============================] - 71s 10ms/sample - loss: 0.9898 - accuracy: 0.6386
Epoch 7/10
6986/6986 [==============================] - 71s 10ms/sample - loss: 0.9267 - accuracy: 0.6663
Epoch 8/10
6986/6986 [==============================] - 71s 10ms/sample - loss: 0.8535 - accuracy: 0.6965
Epoch 9/10
6986/6986 [==============================] - 71s 10ms/sample - loss: 0.8002 - accuracy: 0.7127
Epoch 10/10
6986/6986 [=

In [16]:
model.evaluate(testX,testY)

1746/1746 [==============================] - 7s 4ms/sample - loss: 0.8950 - accuracy: 0.6644


[0.8950177956829093, 0.6643757]